In [2]:
import pandas as pd
from model import Modelo
import random
from statistics import mean

###  Cargando Data, Creación Sets y Parametros

In [ ]:
data = pd.read_csv('data/forecast.csv')

sets = {}
parameters = {}
tiendas ={i:tienda for i, tienda in enumerate(list(data['tienda'].unique()))}
productos ={i:product for i, product in enumerate(list(data['sku'].unique()))}
sets['productos'] = productos
sets['tiendas'] = tiendas

In [ ]:
# como el modelo no considera la dimensionalidad de tiempo, sumare la cantidad total de unidades de todos los productos para cada tienda,
# es decir, tendre una demanda semanal por cada producto en cada tienda
df_pivot = data.pivot_table(
    index='sku',
    columns='tienda',
    values='pred',
    aggfunc='sum',
    fill_value=0 
)
df_pivot

tienda,Tienda_01,Tienda_02,Tienda_03,Tienda_04,Tienda_05,Tienda_06,Tienda_07,Tienda_08
sku,,,,,,,,
SKU_01,6577,5835,6369,4580,7286,4481,4293,3391
SKU_02,3645,4518,3633,6187,4943,3561,3229,3626
SKU_03,2301,2051,1983,3122,1901,2844,1900,1848
SKU_04,1616,1999,2110,1744,2100,2149,1096,1335
SKU_05,1500,1767,1558,1630,1579,1253,1102,784
SKU_06,769,829,545,744,917,788,732,412
SKU_07,2376,2435,1953,2240,2517,2030,1305,1693
SKU_08,1485,1842,2004,1917,1536,1678,1114,842
SKU_09,376,289,334,431,401,368,325,318


### Transformaciones

In [32]:
parameters['DemandaProducto'] = {
    prod_id: df_pivot.loc[prod_name].tolist()
    for prod_id, prod_name in productos.items()
}

# **Supuesto la cantidad mínima sera 1/3 de la demanda
parameters['CantidadMinima'] = {
    prod_id: [int(round(value / 3)) for value in df_pivot.loc[prod_name]]
    for prod_id, prod_name in productos.items()
}

In [33]:
# Costo inventarios numeros aletorios
parameters['CostoInventario'] = {prod_id:{tienda_id:random.randint(5, 10)  for tienda_id in tiendas} for prod_id in productos}
parameters['CostoQuiebre'] = {prod_id:{tienda_id:random.randint(8, 12)  for tienda_id in tiendas} for prod_id in productos}

In [34]:
# **Supuesto** la producción máxima de productos sera 4 veces el promedio de la demanda de todas la tiendas
parameters['ProduccionMaxima'] = {prod_id: int(mean(parameters['DemandaProducto'][prod_id])*4) for prod_id in productos}

# **Supuesto** la capacidad de almacenaje de cada tienda sera 2/3 del total de productos que demandara
parameters['almacenajeTienda'] = {tienda : int(df_pivot[tiendas[tienda]].sum()*2/3) for tienda in tiendas}

### Buscando Óptimo

In [35]:
modelo = Modelo()

modelo.setsParameters(sets, parameters)
modelo.variables()
modelo.objetive()
modelo.constrains()
modelo.run_model()

#############################################
[OPTIMAL] optimal solution profit 792195.0 found


In [ ]:
df_result = []

x = modelo.results()
for i in sets['productos']:
    for j in sets['tiendas']:
        # print(f'Producto {i}, Tienda {j}: {x[i][j].x}')
        value = x[i][j].x
        df_result.append({'producto': i, 'tienda': j, 'cantidad': value})

### Grafico de Resultados, Inventario semanal optimo

In [ ]:
import plotly.express as px

df_result = pd.DataFrame(df_result)

df_result['producto'] = df_result['producto'].map(sets['productos'])  # SKU_01, etc.
df_result['tienda'] = df_result['tienda'].map(sets['tiendas'])        # Tienda_01, etc.

fig = px.bar(
    df_result,
    x='producto',          # eje X: productos
    y='cantidad',          # eje Y: cantidad entregada
    color='tienda',        # color por tienda
    barmode='group',       # barras lado a lado
    title='Cantidad asignada semanalmente por producto y tienda'
)

fig.show()